In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import gridemissions as ge
from gridemissions.viz import set_plots
import pathlib
from ipywidgets import interact

import re
from gridemissions.papers.physics_informed import figure1, figure2, figure3, figure4

COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()
ge.configure_logging("ERROR")

In [ ]:
folder = ge.config["DATA_PATH"] / "EIA_Grid_Monitor" / "processed"
elec_files = [f for f in folder.iterdir() if f.name.endswith("elec.csv")]
raw_files = [f for f in folder.iterdir() if f.name.endswith("raw.csv")]

elec = ge.GraphData(
    pd.concat(
        [pd.read_csv(path, index_col=0, parse_dates=True) for path in elec_files],
        axis=0,
    )
)
raw = ge.GraphData(
    pd.concat(
        [pd.read_csv(path, index_col=0, parse_dates=True) for path in raw_files], axis=0
    )
)
raw.df.sort_index(inplace=True)
elec.df.sort_index(inplace=True)

# Note: we need to drop duplicate indices here because there seems to be duplicates at the beginning/end of the files
elec.df = elec.df[~elec.df.index.duplicated(keep="last")]
raw.df = raw.df[~raw.df.index.duplicated(keep="last")]

In [ ]:
deltas = elec.df - raw.df
delta_quantiles = deltas.quantile([0.1, 0.5, 0.9])
deltas_abs = deltas.abs().quantile([0.1, 0.5, 0.9])
median_raw = raw.df.abs().quantile(0.5)
delta_quantiles_pct = delta_quantiles / median_raw * 100

In [ ]:
n = 13
ba_list = (
    raw.get_data(raw.regions, "D")
    .quantile(0.5)
    .sort_values(ascending=False)
    .index[0 : n * 3]
)
ba_list = [re.split(r"\.|-|_", ba)[1] for ba in ba_list]

# Do one plot for WECC BAs and one for EIC BAs
WECC_BAs = [
    "AVA",
    "AZPS",
    "BANC",
    "BPAT",
    "CHPD",
    "CISO",
    "DEAA",
    "DOPD",
    "EPE",
    "GCPD",
    "GRMA",
    "GWA",
    "HGMA",
    "IID",
    "IPCO",
    "LDWP",
    "NEVP",
    "NWMT",
    "PACE",
    "PACW",
    "PGE",
    "PNM",
    "PSCO",
    "PSEI",
    "SCL",
    "SRP",
    "TEPC",
    "TIDC",
    "TPWR",
    "WACM",
    "WALC",
    "WAUW",
    "WWA",
]
# figure3(ba_list[0:21], deltas_abs, median_raw)
sorted_WECC_BAs = [r for r in ba_list if r in WECC_BAs]
sorted_not_WECC_BAs = [r for r in ba_list if r not in WECC_BAs]

# Summary: Demand, Generation, Total Interchange

In [ ]:
def plot_DGTI(year):
    sel = deltas.index.year == year
    delta_quantiles = deltas.loc[sel].quantile([0.1, 0.5, 0.9])
    deltas_abs = deltas.loc[sel].abs().quantile([0.1, 0.5, 0.9])
    median_raw = raw.df.loc[sel].abs().quantile(0.5)
    f, ax = figure2(list(reversed(ba_list)), n, delta_quantiles, median_raw, raw)
    plt.show()


interact(plot_DGTI, year=range(2018, 2025));

# Summary: Exchanges in the WECC

In [ ]:
def plot_ID_WECC(year):
    sel = deltas.index.year == year
    deltas_abs = deltas.loc[sel].abs().quantile([0.1, 0.5, 0.9])
    median_raw = raw.df.loc[sel].abs().quantile(0.5)
    raw_tmp = ge.GraphData(raw.df.loc[sel])
    f, ax = figure3(sorted_WECC_BAs, deltas_abs, median_raw, raw_tmp)
    plt.show()


interact(plot_ID_WECC, year=range(2018, 2025));

# Summary: Exchanges in the EIC

In [ ]:
def plot_ID_EIC(year):
    sel = deltas.index.year == year
    deltas_abs = deltas.loc[sel].abs().quantile([0.1, 0.5, 0.9])
    median_raw = raw.df.loc[sel].abs().quantile(0.5)
    raw_tmp = ge.GraphData(raw.df.loc[sel])
    f, ax = figure3(sorted_not_WECC_BAs, deltas_abs, median_raw, raw_tmp)
    plt.show()


interact(plot_ID_EIC, year=range(2018, 2025));

# Summary: Generation by source

In [ ]:
def plot_NG_SRC(year):
    nr = 13
    sel = deltas.index.year == year
    deltas_abs = deltas.loc[sel].abs().quantile([0.1, 0.5, 0.9])
    median_raw = raw.df.loc[sel].abs().quantile(0.5)
    raw_tmp = ge.GraphData(raw.df.loc[sel])
    f, ax = figure4(ba_list[0 : 3 * nr], nr, deltas_abs, median_raw, raw)
    plt.show()


interact(plot_NG_SRC, year=range(2018, 2025));